In [15]:
# Dependencies and Setup
import pandas as pd

# File to Load (Remember to change the path if needed.)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read the School Data and Student Data and store into a Pandas DataFrame
school_data_df = pd.read_csv(school_data_to_load)
student_data_df = pd.read_csv(student_data_to_load)

# Cleaning Student Names and Replacing Substrings in a Python String
# Add each prefix and suffix to remove to a list.
prefixes_suffixes = ["Dr. ", "Mr. ","Ms. ", "Mrs. ", "Miss ", " MD", " DDS", " DVM", " PhD"]

# Iterate through the words in the "prefixes_suffixes" list and replace them with an empty space, "".
for word in prefixes_suffixes:
    student_data_df["student_name"] = student_data_df["student_name"].str.replace(word,"")

# Check names.
student_data_df.head(10)

## Deliverable 1: Replace the reading and math scores.

### Replace the 9th grade reading and math scores at Thomas High School with NaN.

# Install numpy using conda install numpy or pip install numpy. 
# Step 1. Import numpy as np.
import numpy as np

# Step 2. Use the loc method on the student_data_df to select all the reading scores from the 9th grade at Thomas High School and replace them with NaN.
student_data_df.loc[(student_data_df["grade"]=="9th") &(student_data_df["school_name"]=="Thomas High School"),["reading_score"]]=np.nan



#  Step 3. Refactor the code in Step 2 to replace the math scores with NaN.
student_data_df.loc[(student_data_df["grade"]=="9th") &(student_data_df["school_name"]=="Thomas High School"),["math_score"]]=np.nan

#  Step 4. Check the student data for NaN's. 

student_data_df.count()

## Deliverable 2 : Repeat the school district analysis

### District Summary

# Combine the data into a single dataset
school_data_complete_df = pd.merge(student_data_df, school_data_df, how="left", on=["school_name", "school_name"])
school_data_complete_df.head()

# Calculate the Totals (Schools and Students)
school_count = len(school_data_complete_df["school_name"].unique())
student_count = school_data_complete_df["Student ID"].count()

# Calculate the Total Budget
total_budget = school_data_df["budget"].sum()

# Calculate the Average Scores using the "clean_student_data".
average_reading_score = school_data_complete_df["reading_score"].mean()
average_math_score = school_data_complete_df["math_score"].mean()

# Step 1. Get the number of students that are in ninth grade at Thomas High School.
# These students have no grades. 
no_grades=school_data_complete_df.loc[(school_data_complete_df["grade"]=="9th") &(student_data_df["school_name"]=="Thomas High School")].count()['student_name']

# Get the total student count 
student_count = school_data_complete_df["Student ID"].count()


# Step 2. Subtract the number of students that are in ninth grade at 
# Thomas High School from the total student count to get the new total student count.
new_student_count=student_count-no_grades


# Calculate the passing rates using the "clean_student_data".
passing_math_count = school_data_complete_df[(school_data_complete_df["math_score"] >= 70)].count()["student_name"]
passing_reading_count = school_data_complete_df[(school_data_complete_df["reading_score"] >= 70)].count()["student_name"]



# Step 3. Calculate the passing percentages with the new total student count.
passing_math_percentage=passing_math_count/float(new_student_count)*100
passing_reading_percentage=passing_reading_count/float(new_student_count)*100


# Calculate the students who passed both reading and math.
passing_math_reading = school_data_complete_df[(school_data_complete_df["math_score"] >= 70)
                                               & (school_data_complete_df["reading_score"] >= 70)]

# Calculate the number of students that passed both reading and math.
overall_passing_math_reading_count = passing_math_reading["student_name"].count()


# Step 4.Calculate the overall passing percentage with new total student count.
overall_passing_percentage=overall_passing_math_reading_count/float(new_student_count)*100

# Create a DataFrame
district_summary_df = pd.DataFrame(
          [{"Total Schools": school_count, 
          "Total Students":new_student_count, 
          "Total Budget": total_budget,
          "Average Math Score": average_math_score, 
          "Average Reading Score": average_reading_score,
          "% Passing Math": passing_math_percentage,
         "% Passing Reading": passing_reading_percentage,
        "% Overall Passing": overall_passing_percentage}])



# Format the "Total Students" to have the comma for a thousands separator.
district_summary_df["Total Students"] = district_summary_df["Total Students"].map("{:,}".format)
# Format the "Total Budget" to have the comma for a thousands separator, a decimal separator and a "$".
district_summary_df["Total Budget"] = district_summary_df["Total Budget"].map("${:,.2f}".format)
# Format the columns.
district_summary_df["Average Math Score"] = district_summary_df["Average Math Score"].map("{:.1f}".format)
district_summary_df["Average Reading Score"] = district_summary_df["Average Reading Score"].map("{:.1f}".format)
district_summary_df["% Passing Math"] = district_summary_df["% Passing Math"].map("{:.1f}".format)
district_summary_df["% Passing Reading"] = district_summary_df["% Passing Reading"].map("{:.1f}".format)
district_summary_df["% Overall Passing"] = district_summary_df["% Overall Passing"].map("{:.1f}".format)

# Display the data frame
district_summary_df

##  School Summary

# Determine the School Type
per_school_types = school_data_df.set_index(["school_name"])["type"]

# Calculate the total student count.
per_school_counts = school_data_complete_df["school_name"].value_counts()

# Calculate the total school budget and per capita spending
per_school_budget = school_data_complete_df.groupby(["school_name"]).mean()["budget"]
# Calculate the per capita spending.
per_school_capita = per_school_budget / per_school_counts

# Calculate the average test scores.
per_school_math = school_data_complete_df.groupby(["school_name"]).mean()["math_score"]
per_school_reading = school_data_complete_df.groupby(["school_name"]).mean()["reading_score"]

# Calculate the passing scores by creating a filtered DataFrame.
per_school_passing_math = school_data_complete_df[(school_data_complete_df["math_score"] >= 70)]
per_school_passing_reading = school_data_complete_df[(school_data_complete_df["reading_score"] >= 70)]

# Calculate the number of students passing math and passing reading by school.
per_school_passing_math = per_school_passing_math.groupby(["school_name"]).count()["student_name"]
per_school_passing_reading = per_school_passing_reading.groupby(["school_name"]).count()["student_name"]

# Calculate the percentage of passing math and reading scores per school.
per_school_passing_math = per_school_passing_math / per_school_counts * 100
per_school_passing_reading = per_school_passing_reading / per_school_counts * 100

# Calculate the students who passed both reading and math.
per_passing_math_reading = school_data_complete_df[(school_data_complete_df["reading_score"] >= 70)
                                               & (school_data_complete_df["math_score"] >= 70)]

# Calculate the number of students passing math and passing reading by school.
per_passing_math_reading = per_passing_math_reading.groupby(["school_name"]).count()["student_name"]

# Calculate the percentage of passing math and reading scores per school.
per_overall_passing_percentage = per_passing_math_reading / per_school_counts * 100

# Create the DataFrame
per_school_summary_df = pd.DataFrame({
    "School Type": per_school_types,
    "Total Students": per_school_counts,
    "Total School Budget": per_school_budget,
    "Per Student Budget": per_school_capita,
    "Average Math Score": per_school_math,
    "Average Reading Score": per_school_reading,
    "% Passing Math": per_school_passing_math,
    "% Passing Reading": per_school_passing_reading,
    "% Overall Passing": per_overall_passing_percentage})


#per_school_summary_df.head()

# Format the Total School Budget and the Per Student Budget
per_school_summary_df["Total School Budget"] = per_school_summary_df["Total School Budget"].map("${:,.2f}".format)
per_school_summary_df["Per Student Budget"] = per_school_summary_df["Per Student Budget"].map("${:,.2f}".format)

# Display the data frame
per_school_summary_df


# Step 5.  Get the number of 10th-12th graders from Thomas High School (THS).

studets_ths=school_data_complete_df[(school_data_complete_df["grade"] != "9th") &(school_data_complete_df["school_name"]=="Thomas High School")]
count_studets_ths=school_data_complete_df[(school_data_complete_df["grade"] != "9th") &(school_data_complete_df["school_name"]=="Thomas High School")].count()["Student ID"]

# Step 6. Get all the students passing math from THS
THS_passing_maths=school_data_complete_df.loc[(school_data_complete_df["grade"]!="9th") &(student_data_df["school_name"]=="Thomas High School")&(student_data_df["math_score"]>=70)]

passing_math_count=THS_passing_maths.count()["student_name"]


# Step 7. Get all the students passing reading from THS
THS_passing_reading=school_data_complete_df.loc[(school_data_complete_df["grade"]!="9th") &(student_data_df["school_name"]=="Thomas High School")&(student_data_df["reading_score"]>=70)]
passing_reading_count=THS_passing_reading.count()["student_name"]

# Step 8. Get all the students passing math and reading from THS
THS_math_reading_passing=school_data_complete_df.loc[(school_data_complete_df["grade"]!="9th") &(student_data_df["school_name"]=="Thomas High School")&(student_data_df["math_score"]>=70)&(student_data_df["reading_score"]>=70)]
passing_math_reading_count=THS_math_reading_passing.count()["student_name"]

# Step 9. Calculate the percentage of 10th-12th grade students passing math from Thomas High School. 
passing_percentage_math=passing_math_count/float(count_studets_ths)*100


# Step 10. Calculate the percentage of 10th-12th grade students passing reading from Thomas High School.
passing_percentage_reading=passing_reading_count/float(count_studets_ths)*100

# Step 11. Calculate the overall passing percentage of 10th-12th grade from Thomas High School. 
overall_passing_percentage=passing_math_reading_count/float(count_studets_ths)*100


# Step 12. Replace the passing math percent for Thomas High School in the per_school_summary_df.
per_school_summary_df.loc['Thomas High School','% Passing Math']=passing_percentage_math


# Step 13. Replace the passing reading percentage for Thomas High School in the per_school_summary_df.
per_school_summary_df.loc['Thomas High School','% Passing Reading']=passing_percentage_reading

# Step 14. Replace the overall passing percentage for Thomas High School in the per_school_summary_df.
per_school_summary_df.loc['Thomas High School','% Overall Passing']=overall_passing_percentage

#per_school_summary_df

## High and Low Performing Schools 

# Sort and show top five schools.
top_schools=per_school_summary_df.sort_values("% Overall Passing",ascending=False)
top_schools.head()

# Sort and show bottom five schools.
bottom_schools=per_school_summary_df.sort_values("% Overall Passing")
bottom_schools.head()

## Math and Reading Scores by Grade

# Create a Series of scores by grade levels using conditionals.
ninth_graders = school_data_complete_df[(school_data_complete_df["grade"] == "9th")]

tenth_graders = school_data_complete_df[(school_data_complete_df["grade"] == "10th")]

eleventh_graders = school_data_complete_df[(school_data_complete_df["grade"] == "11th")]

twelfth_graders = school_data_complete_df[(school_data_complete_df["grade"] == "12th")]

# Group each school Series by the school name for the average math score.
ninth_grade_math_scores = ninth_graders.groupby(["school_name"]).mean()["math_score"]

tenth_grade_math_scores = tenth_graders.groupby(["school_name"]).mean()["math_score"]

eleventh_grade_math_scores = eleventh_graders.groupby(["school_name"]).mean()["math_score"]

twelfth_grade_math_scores = twelfth_graders.groupby(["school_name"]).mean()["math_score"]

# Group each school Series by the school name for the average reading score.
ninth_grade_reading_scores = ninth_graders.groupby(["school_name"]).mean()["reading_score"]

tenth_grade_reading_scores = tenth_graders.groupby(["school_name"]).mean()["reading_score"]

eleventh_grade_reading_scores = eleventh_graders.groupby(["school_name"]).mean()["reading_score"]

twelfth_grade_reading_scores = twelfth_graders.groupby(["school_name"]).mean()["reading_score"]

# Combine each Series for average math scores by school into single data frame.
math_scores_by_grade = pd.DataFrame({
               "9th": ninth_grade_math_scores,
               "10th": tenth_grade_math_scores,
               "11th": eleventh_grade_math_scores,
               "12th": twelfth_grade_math_scores})



# Combine each Series for average reading scores by school into single data frame.
reading_scores_by_grade = pd.DataFrame({
              "9th": ninth_grade_reading_scores,
              "10th": tenth_grade_reading_scores,
              "11th": eleventh_grade_reading_scores,
              "12th": twelfth_grade_reading_scores})



# Format each grade column.
math_scores_by_grade["9th"]=math_scores_by_grade["9th"].map("{:.2f}".format)
math_scores_by_grade["10th"]=math_scores_by_grade["10th"].map("{:.2f}".format)
math_scores_by_grade["11th"]=math_scores_by_grade["11th"].map("{:.2f}".format)
math_scores_by_grade["12th"]=math_scores_by_grade["12th"].map("{:.2f}".format)
reading_scores_by_grade["9th"]=reading_scores_by_grade["9th"].map("{:.2f}".format)
reading_scores_by_grade["10th"]=reading_scores_by_grade["10th"].map("{:.2f}".format)
reading_scores_by_grade["11th"]=reading_scores_by_grade["11th"].map("{:.2f}".format)
reading_scores_by_grade["12th"]=reading_scores_by_grade["12th"].map("{:.2f}".format)

# Remove the index.
math_scores_by_grade.index.name= None


# Display the data frame
math_scores_by_grade

## Remove the index.
reading_scores_by_grade.index.name =None

# Display the data frame
reading_scores_by_grade

## Scores by School Spending

# Establish the spending bins and group names.

spending_bins = [0, 585, 630, 645, 675]
group_names = ["<$584", "$585-629", "$630-644", "$645-675"]
# Categorize spending based on the bins.
pd.cut(per_school_capita, spending_bins)

# Calculate averages for the desired columns. 
per_school_summary_df["Spending Ranges (Per Student)"] = pd.cut(per_school_capita, spending_bins, labels=group_names)
spending_math_scores = per_school_summary_df.groupby(["Spending Ranges (Per Student)"]).mean()["Average Math Score"]

spending_reading_scores = per_school_summary_df.groupby(["Spending Ranges (Per Student)"]).mean()["Average Reading Score"]

spending_passing_math = per_school_summary_df.groupby(["Spending Ranges (Per Student)"]).mean()["% Passing Math"]

spending_passing_reading = per_school_summary_df.groupby(["Spending Ranges (Per Student)"]).mean()["% Passing Reading"]

overall_passing_spending = per_school_summary_df.groupby(["Spending Ranges (Per Student)"]).mean()["% Overall Passing"]

# Create the DataFrame
spending_summary_df = pd.DataFrame({
          "Average Math Score" : spending_math_scores,
          "Average Reading Score": spending_reading_scores,
          "% Passing Math": spending_passing_math,
          "% Passing Reading": spending_passing_reading,
          "% Overall Passing": overall_passing_spending})

# Format the DataFrame 
spending_summary_df["Average Math Score"]=spending_summary_df["Average Math Score"].map("{:.1f}".format)
spending_summary_df["Average Reading Score"]=spending_summary_df["Average Reading Score"].map("{:.1f}".format)
spending_summary_df["% Passing Math"] = spending_summary_df["% Passing Math"].map("{:.0f}".format)

spending_summary_df["% Passing Reading"] = spending_summary_df["% Passing Reading"].map("{:.0f}".format)

spending_summary_df["% Overall Passing"] = spending_summary_df["% Overall Passing"].map("{:.0f}".format)
spending_summary_df

## Scores by School Size

# Establish the bins.
size_bins=[0,1000,1999,5000]
group_names = ["Small", "Medium", "Large"]
# Categorize spending based on the bins.
per_school_summary_df["School Size"] = pd.cut(per_school_summary_df["Total Students"], size_bins, labels=group_names)

# Calculate averages for the desired columns. 
size_math_scores=per_school_summary_df.groupby('School Size').mean()["Average Math Score"]
size_reading_scores = per_school_summary_df.groupby(["School Size"]).mean()["Average Reading Score"]

size_passing_math = per_school_summary_df.groupby(["School Size"]).mean()["% Passing Math"]

size_passing_reading = per_school_summary_df.groupby(["School Size"]).mean()["% Passing Reading"]

size_overall_passing = per_school_summary_df.groupby(["School Size"]).mean()["% Overall Passing"]

# Assemble into DataFrame. 
size_summary_df=pd.DataFrame({
          "Average Math Score" : size_math_scores,
          "Average Reading Score": size_reading_scores,
          "% Passing Math": size_passing_math ,
          "% Passing Reading": size_passing_reading,
          "% Overall Passing": size_overall_passing})

# Format the DataFrame  
size_summary_df["Average Math Score"] = size_summary_df["Average Math Score"].map("{:.1f}".format)

size_summary_df["Average Reading Score"] = size_summary_df["Average Reading Score"].map("{:.1f}".format)

size_summary_df["% Passing Math"] = size_summary_df["% Passing Math"].map("{:.0f}".format)

size_summary_df["% Passing Reading"] = size_summary_df["% Passing Reading"].map("{:.0f}".format)

size_summary_df["% Overall Passing"] = size_summary_df["% Overall Passing"].map("{:.0f}".format)

size_summary_df

## Scores by School Type

# Calculate averages for the desired columns. 
type_math_scores=per_school_summary_df.groupby('School Type').mean()["Average Math Score"]
type_reading_scores = per_school_summary_df.groupby(["School Type"]).mean()["Average Reading Score"]

type_passing_math = per_school_summary_df.groupby(["School Type"]).mean()["% Passing Math"]

type_passing_reading = per_school_summary_df.groupby(["School Type"]).mean()["% Passing Reading"]

type_overall_passing = per_school_summary_df.groupby(["School Type"]).mean()["% Overall Passing"]

# Assemble into DataFrame. 
type_summary_df=pd.DataFrame({
          "Average Math Score" : type_math_scores,
          "Average Reading Score": type_reading_scores,
          "% Passing Math": type_passing_math ,
          "% Passing Reading": type_passing_reading,
          "% Overall Passing": type_overall_passing})

# # Format the DataFrame 
type_summary_df["Average Math Score"] = type_summary_df["Average Math Score"].map("{:.1f}".format)

type_summary_df["Average Reading Score"] = type_summary_df["Average Reading Score"].map("{:.1f}".format)

type_summary_df["% Passing Math"] = type_summary_df["% Passing Math"].map("{:.0f}".format)

type_summary_df["% Passing Reading"] = type_summary_df["% Passing Reading"].map("{:.0f}".format)

type_summary_df["% Overall Passing"] = type_summary_df["% Overall Passing"].map("{:.0f}".format)

type_summary_df

################################################################################################################

#### below is the practice code 



import pandas as pd

school_data_df=pd.read_csv("Resources/schools_complete.csv")
students_complete_df=pd.read_csv("Resources/students_complete.csv")

students_complete_df.head()




prefixes_suffixes = ["Dr. ", "Mr. ","Ms. ", "Mrs. ", "Miss ", " MD", " DDS", " DVM", " PhD"]
for name in prefixes_suffixes:
    students_complete_df["student_name"] = students_complete_df["student_name"].str.replace(name,"")
    
students_complete_df

students_complete_df.head()

student_data_df=students_complete_df


school_data_df.head()

school_data_complete_df=pd.merge(student_data_df,school_data_df,on=['school_name','school_name'])

school_data_complete_df.head()

school_data_df.rename(columns={'school_name1':'school_name'})

student_count = school_data_complete_df.count()
student_count

school_data_complete_df['Student ID'].count()

school_count = school_data_df["school_name"].count()
school_count

@school_count = school_data_complete_df["school_name"].unique()


total_budget=school_data_df['budget'].sum()
total_budget

#reading_score 	math_score
average_reading_score=school_data_complete_df['reading_score'].mean()
average_reading_score

average_math_score=school_data_complete_df['math_score'].mean()
average_math_score

passing_math=school_data_complete_df['math_score']>=70
passing_reading=school_data_complete_df['reading_score']>=70
passing_math

passing_math=school_data_complete_df[passing_math]

passing_reading

passing_reading = school_data_complete_df[school_data_complete_df["reading_score"] >= 70]
passing_reading

passing_math = school_data_complete_df[school_data_complete_df["math_score"] >= 70]
passing_math.head()

passing_math_count=passing_math['student_name'].count()
passing_math_count

passing_reading_count=passing_reading['student_name'].count()
passing_reading_count

student_count=school_data_complete_df["Student ID"].count()

student_count

student_count.dtype

passing_math_percentage=passing_math_count/float(student_count)*100
passing_math_percentage.dtype

passing_reading__percentage=passing_reading_count/float(student_count)*100
passing_reading__percentage

#passin_math_reading=school_data_complete_df[school_data_complete_df["math_score"] >= 70 & ['reading_score']>=70]

passing_math_reading=school_data_complete_df[(school_data_complete_df["math_score"] >= 70)  & (school_data_complete_df["reading_score"] >=70)]
passing_math_reading.student_name.count()

overall_passing_math_reading_count = passing_math_reading["student_name"].count()
overall_passing_math_reading_count

overall_passing_percentage=overall_passing_math_reading_count/float(student_count)*100
overall_passing_percentage

student_count

district_summary_df=pd.DataFrame([{"Total Schools":school_count,"Total Students":student_count,"Total Budget":total_budget ,
                                  "Average Math Score": average_math_score,
          "Average Reading Score": average_reading_score,
          "% Passing Math": passing_math_percentage,
         "% Passing Reading": passing_reading__percentage,
        "% Overall Passing": overall_passing_percentage}])
district_summary_df

district_summary_df["Average Math Score"]=district_summary_df["Average Math Score"].map("{:.2f}".format)

district_summary_df

district_summary_df["Total Students"]=district_summary_df["Total Students"].map("{:,}".format)
district_summary_df

district_summary_df["Total Budget"]=district_summary_df["Total Budget"].map("${:,.2f}".format)
district_summary_df

district_summary_df["Average Reading Score"]=district_summary_df["Average Reading Score"].map("{:.2f}".format)

district_summary_df                                                                             

#district_summary_df["% Passing Math"]=district_summary_df["% Passing Math"].map("{:.0f}".format)
#district_summary_df["% Passing Math"] = district_summary_df["% Passing Math"].map("{:.0f}".format)
district_summary_df.dtypes

# Format the columns.
district_summary_df["Average Math Score"] = district_summary_df["Average Math Score"].map("{:.1f}".format)

district_summary_df["Average Reading Score"] = district_summary_df["Average Reading Score"].map("{:.1f}".format)

district_summary_df["% Passing Math"] = district_summary_df["% Passing Math"].map("{:.0f}".format)

district_summary_df["% Passing Reading"] = district_summary_df["% Passing Reading"].map("{:.0f}".format)

district_summary_df["% Overall Passing"] = district_summary_df["% Overall Passing"].map("{:.0f}".format)

district_summary_df

new_order=["Total Schools", "Total Students", "Total Budget","Average Math Score", "Average Reading Score", "% Passing Math", "% Passing Reading", "% Overall Passing"]
district_summary_df=district_summary_df[new_order]
district_summary_df

#district_summary_df=district_summary_df["Total Schools", "Total Students", "Total Budget","Average Math Score", "Average Reading Score", "% Passing Math", "% Passing Reading", "% Overall Passing"]

school_data_complete_df.head()


school_data_complete_df.index

school_data_df.head()

per_school_types =school_data_df.set_index(["school_name"])["type"]
per_school_types

df = pd.DataFrame(per_school_types)
df

per_school_counts = school_data_df["size"]
per_school_counts

per_school_counts = school_data_df.set_index(["school_name"])["size"]
per_school_counts

school_data_complete_df.head()

school_data_complete_df.value_counts("school_name")

per_school_counts = school_data_complete_df["school_name"].value_counts()
per_school_counts

per_school_budget=school_data_df.set_index(["school_name"])["budget"]
per_school_budget

per_school_capita = per_school_budget / per_school_counts
per_school_capita

student_data_df.head()

average=student_data_df.set_index(["school_name"])['reading_score']
average



per_school_reading=school_data_complete_df.groupby(['school_name']).mean()["reading_score"]
per_school_reading

per_school_math = school_data_complete_df.groupby(["school_name"]).mean()["math_score"]
per_school_math

per_school_passing_math = school_data_complete_df[(school_data_complete_df["math_score"] >= 70)]

per_school_passing_reading = school_data_complete_df[(school_data_complete_df["reading_score"] >= 70)]
per_passing_math_reading =school_data_complete_df[(school_data_complete_df["reading_score"] >= 70)&(school_data_complete_df["math_score"] >= 70)]

per_school_passing_math=per_school_passing_math.groupby(["school_name"]).count()["student_name"]

per_school_passing_reading=per_school_passing_reading.groupby(["school_name"]).count()['student_name']
per_passing_math_reading=per_passing_math_reading.groupby(["school_name"]).count()['student_name']

per_school_passing_math_percentage=per_school_passing_math/per_school_counts*100

per_school_passing_math_percentage=per_school_passing_math_percentage.map("{:.2f}".format)
per_school_passing_reading_percentage=per_school_passing_reading/per_school_counts*100

per_school_passing_reading_percentage=per_school_passing_reading_percentage.map("{:.2f}".format)
per_passing_math_reading_percentage=per_passing_math_reading/per_school_counts*100
per_passing_math_reading_percentage=per_passing_math_reading_percentage.map("{:.0f}".format)
per_passing_math_reading_percentage

per_school_counts


#per_passing_math_reading_percentage=per_passing_math_reading_percentage.map("{:2f}".format)


per_passing_math_reading_percentage

per_school_summary_df = pd.DataFrame({
             "School Type": per_school_types,
             "Total Students": per_school_counts,
             "Total School Budget": per_school_budget,
             "Per Student Budget": per_school_capita,
             "Average Math Score": per_school_math,
           "Average Reading Score": per_school_reading,
           "% Passing Math": per_school_passing_math_percentage,
           "% Passing Reading": per_school_passing_reading_percentage,
           "% Overall Passing": per_passing_math_reading_percentage})
per_school_summary_df.head()

per_school_summary_df["Total School Budget"]=per_school_summary_df["Total School Budget"].map("${:,.2f}".format)
per_school_summary_df["Average Math Score"]=per_school_summary_df["Average Math Score"].map("{:.2f}".format)
per_school_summary_df["Average Reading Score"]=per_school_summary_df["Average Reading Score"].map("{:.2f}".format)
per_school_summary_df["Per Student Budget"]=per_school_summary_df["Per Student Budget"].map("${:.2f}".format)




per_school_summary_df.head()


top_schools=per_school_summary_df.sort_values("% Overall Passing",ascending=False)
top_schools.head()

bottom_schools=per_school_summary_df.sort_values(["% Overall Passing"])
bottom_schools.head()

school_data_complete_df.head()

grade=school_data_complete_df.groupby("school_name")["grade"]

grade

ninth_graders = school_data_complete_df[(school_data_complete_df["grade"] == "9th")]

tenth_graders = school_data_complete_df[(school_data_complete_df["grade"] == "10th")]

eleventh_graders = school_data_complete_df[(school_data_complete_df["grade"] == "11th")]

twelfth_graders = school_data_complete_df[(school_data_complete_df["grade"] == "12th")]

ninth_grade_math_scores = ninth_graders.groupby(["school_name"]).mean()["math_score"]

tenth_grade_math_scores = tenth_graders.groupby(["school_name"]).mean()["math_score"]

eleventh_grade_math_scores = eleventh_graders.groupby(["school_name"]).mean()["math_score"]

twelfth_grade_math_scores = twelfth_graders.groupby(["school_name"]).mean()["math_score"]

eleventh_grade_math_scores

ninth_grade_reading_scores = ninth_graders.groupby(["school_name"]).mean()["reading_score"]

tenth_grade_reading_scores = tenth_graders.groupby(["school_name"]).mean()["reading_score"]

eleventh_grade_reading_scores = eleventh_graders.groupby(["school_name"]).mean()["reading_score"]

twelfth_grade_reading_scores = twelfth_graders.groupby(["school_name"]).mean()["reading_score"]


math_scores_by_grade = pd.DataFrame({
               "9th": ninth_grade_math_scores,
               "10th": tenth_grade_math_scores,
               "11th": eleventh_grade_math_scores,
               "12th": twelfth_grade_math_scores})

math_scores_by_grade["9th"]=math_scores_by_grade["9th"].map("{:.2f}".format)
math_scores_by_grade["10th"]=math_scores_by_grade["10th"].map("{:.2f}".format)
math_scores_by_grade["11th"]=math_scores_by_grade["11th"].map("{:.2f}".format)
math_scores_by_grade["12th"]=math_scores_by_grade["12th"].map("{:.2f}".format)

math_scores_by_grade.head()
                      
                     






reading_scores_by_grade = pd.DataFrame({
              "9th": ninth_grade_reading_scores,
              "10th": tenth_grade_reading_scores,
              "11th": eleventh_grade_reading_scores,
              "12th": twelfth_grade_reading_scores})

reading_scores_by_grade["9th"]=reading_scores_by_grade["9th"].map("{:.2f}".format)
reading_scores_by_grade["10th"]=reading_scores_by_grade["10th"].map("{:.2f}".format)
reading_scores_by_grade["11th"]=reading_scores_by_grade["11th"].map("{:.2f}".format)
reading_scores_by_grade["12th"]=reading_scores_by_grade["12th"].map("{:.2f}".format)

reading_scores_by_grade.head()

math_scores_by_grade.index.name= None
reading_scores_by_grade.index.name =None

reading_scores_by_grade.head()

math_scores_by_grade

per_school_capita

per_school_capita.describe()

spending_bins = [0, 585, 630, 645, 675]

spending_bins

pd.cut(per_school_capita, spending_bins)

per_school_capita.groupby(pd.cut(per_school_capita, spending_bins)).count()

spending_bins = [0, 585, 630, 645, 675]
group_names = ["<$584", "$585-629", "$630-644", "$645-675"]

per_school_summary_df.head()

per_school_summary_df["Spending Ranges (Per Student)"] = pd.cut(per_school_capita, spending_bins, labels=group_names)

per_school_summary_df

per_school_summary_df["Average Math Score"]=pd.to_numeric(per_school_summary_df["Average Math Score"])
per_school_summary_df["Average Reading Score"]=pd.to_numeric(per_school_summary_df["Average Reading Score"])
per_school_summary_df["% Passing Math"]=pd.to_numeric(per_school_summary_df["% Passing Math"])
per_school_summary_df["% Passing Reading"]=pd.to_numeric(per_school_summary_df["% Passing Reading"])
per_school_summary_df["% Overall Passing"]=pd.to_numeric(per_school_summary_df["% Overall Passing"])


spending_math_scores = per_school_summary_df.groupby(["Spending Ranges (Per Student)"]).mean()["Average Math Score"]

spending_reading_scores = per_school_summary_df.groupby(["Spending Ranges (Per Student)"]).mean()["Average Reading Score"]

spending_passing_math = per_school_summary_df.groupby(["Spending Ranges (Per Student)"]).mean()["% Passing Math"]

spending_passing_reading = per_school_summary_df.groupby(["Spending Ranges (Per Student)"]).mean()["% Passing Reading"]

overall_passing_spending = per_school_summary_df.groupby(["Spending Ranges (Per Student)"]).mean()["% Overall Passing"]

overall_passing_spending

spending_summary_df = pd.DataFrame({
          "Average Math Score" : spending_math_scores,
          "Average Reading Score": spending_reading_scores,
          "% Passing Math": spending_passing_math,
          "% Passing Reading": spending_passing_reading,
          "% Overall Passing": overall_passing_spending})

spending_summary_df


spending_summary_df["Average Math Score"]=spending_summary_df["Average Math Score"].map("{:.1f}".format)
spending_summary_df["Average Reading Score"]=spending_summary_df["Average Reading Score"].map("{:.1f}".format)
spending_summary_df["% Passing Math"] = spending_summary_df["% Passing Math"].map("{:.0f}".format)

spending_summary_df["% Passing Reading"] = spending_summary_df["% Passing Reading"].map("{:.0f}".format)

spending_summary_df["% Overall Passing"] = spending_summary_df["% Overall Passing"].map("{:.0f}".format)


spending_summary_df

per_school_counts.describe()


school_size=[0,1000,1999,5000]
group_names = ["Small", "Medium", "Large"]
pd.cut(per_school_summary_df, school_size)

per_school_summary_df["School size)"] = pd.cut(per_school_summary_df, school_size, labels=group_names)

per_school_summary_df

size_bins = [0, 1000, 2000, 5000]
group_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]
per_school_summary_df["School Size"] = pd.cut(per_school_summary_df["Total Students"], size_bins, labels=group_names)

per_school_summary_df.head()

size_math_scores=per_school_summary_df.groupby('School Size').mean()["Average Math Score"]
size_reading_scores = per_school_summary_df.groupby(["School Size"]).mean()["Average Reading Score"]

size_passing_math = per_school_summary_df.groupby(["School Size"]).mean()["% Passing Math"]

size_passing_reading = per_school_summary_df.groupby(["School Size"]).mean()["% Passing Reading"]

size_overall_passing = per_school_summary_df.groupby(["School Size"]).mean()["% Overall Passing"]

size_summary_df=pd.DataFrame({
          "Average Math Score" : size_math_scores,
          "Average Reading Score": size_reading_scores,
          "% Passing Math": size_passing_math ,
          "% Passing Reading": size_passing_reading,
          "% Overall Passing": size_overall_passing})
size_summary_df

# Formatting.
size_summary_df["Average Math Score"] = size_summary_df["Average Math Score"].map("{:.1f}".format)

size_summary_df["Average Reading Score"] = size_summary_df["Average Reading Score"].map("{:.1f}".format)

size_summary_df["% Passing Math"] = size_summary_df["% Passing Math"].map("{:.0f}".format)

size_summary_df["% Passing Reading"] = size_summary_df["% Passing Reading"].map("{:.0f}".format)

size_summary_df["% Overall Passing"] = size_summary_df["% Overall Passing"].map("{:.0f}".format)

size_summary_df

per_school_summary_df.head()


type_math_scores=per_school_summary_df.groupby('School Type').mean()["Average Math Score"]
type_reading_scores = per_school_summary_df.groupby(["School Type"]).mean()["Average Reading Score"]

type_passing_math = per_school_summary_df.groupby(["School Type"]).mean()["% Passing Math"]

type_passing_reading = per_school_summary_df.groupby(["School Type"]).mean()["% Passing Reading"]

type_overall_passing = per_school_summary_df.groupby(["School Type"]).mean()["% Overall Passing"]

type_summary_df=pd.DataFrame({
          "Average Math Score" : type_math_scores,
          "Average Reading Score": type_reading_scores,
          "% Passing Math": type_passing_math ,
          "% Passing Reading": type_passing_reading,
          "% Overall Passing": type_overall_passing})
type_summary_df.dtypes

type_summary_df["Average Math Score"] = type_summary_df["Average Math Score"].map("{:.1f}".format)

type_summary_df["Average Reading Score"] = type_summary_df["Average Reading Score"].map("{:.1f}".format)

type_summary_df["% Passing Math"] = type_summary_df["% Passing Math"].map("{:.0f}".format)

type_summary_df["% Passing Reading"] = type_summary_df["% Passing Reading"].map("{:.0f}".format)

type_summary_df["% Overall Passing"] = type_summary_df["% Overall Passing"].map("{:.0f}".format)


type_summary_df.dtypes

type_summary_df

# Dependencies and Setup
import pandas as pd

# File to Load (Remember to change the path if needed.)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read the School Data and Student Data and store into a Pandas DataFrame
school_data_df = pd.read_csv(school_data_to_load)
student_data_df = pd.read_csv(student_data_to_load)

# Cleaning Student Names and Replacing Substrings in a Python String
# Add each prefix and suffix to remove to a list.
prefixes_suffixes = ["Dr. ", "Mr. ","Ms. ", "Mrs. ", "Miss ", " MD", " DDS", " DVM", " PhD"]

# Iterate through the words in the "prefixes_suffixes" list and replace them with an empty space, "".
for word in prefixes_suffixes:
    student_data_df["student_name"] = student_data_df["student_name"].str.replace(word,"")

# Check names.
student_data_df.head(10)

C:\Users\aryam\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:18: FutureWarning: The default value of regex will change from True to False in a future version.


,Student ID,student_name,gender,grade,school_name,reading_score,math_score
0,0,Paul Bradley,M,9th,Huang High School,66,79
1,1,Victor Smith,M,12th,Huang High School,94,61
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60
3,3,Richard Scott,M,12th,Huang High School,67,58
4,4,Bonnie Ray,F,9th,Huang High School,97,84
5,5,Bryan Miranda,M,9th,Huang High School,94,94
6,6,Sheena Carter,F,11th,Huang High School,82,80
7,7,Nicole Baker,F,12th,Huang High School,96,69
8,8,Michael Roth,M,10th,Huang High School,95,87
9,9,Matthew Greene,M,10th,Huang High School,96,84


## Deliverable 1: Replace the reading and math scores.

### Replace the 9th grade reading and math scores at Thomas High School with NaN.

In [16]:
# Install numpy using conda install numpy or pip install numpy. 
# Step 1. Import numpy as np.
import numpy as np

In [17]:
# Step 2. Use the loc method on the student_data_df to select all the reading scores from the 9th grade at Thomas High School and replace them with NaN.
student_data_df.loc[(student_data_df["grade"]=="9th") &(student_data_df["school_name"]=="Thomas High School"),["reading_score"]]=np.nan



In [18]:
#  Step 3. Refactor the code in Step 2 to replace the math scores with NaN.
student_data_df.loc[(student_data_df["grade"]=="9th") &(student_data_df["school_name"]=="Thomas High School"),["math_score"]]=np.nan

In [19]:
#  Step 4. Check the student data for NaN's. 

student_data_df

,Student ID,student_name,gender,grade,school_name,reading_score,math_score
0,0,Paul Bradley,M,9th,Huang High School,66.0,79.0
1,1,Victor Smith,M,12th,Huang High School,94.0,61.0
2,2,Kevin Rodriguez,M,12th,Huang High School,90.0,60.0
3,3,Richard Scott,M,12th,Huang High School,67.0,58.0
4,4,Bonnie Ray,F,9th,Huang High School,97.0,84.0
...,...,...,...,...,...,...,...
39165,39165,Donna Howard,F,12th,Thomas High School,99.0,90.0
39166,39166,Dawn Bell,F,10th,Thomas High School,95.0,70.0
39167,39167,Rebecca Tanner,F,9th,Thomas High School,NaN,NaN
39168,39168,Desiree Kidd,F,10th,Thomas High School,99.0,90.0


In [20]:
student_data_df.count()

Student ID       39170
student_name     39170
gender           39170
grade            39170
school_name      39170
reading_score    38709
math_score       38709
dtype: int64

## Deliverable 2 : Repeat the school district analysis

### District Summary

In [21]:
# Combine the data into a single dataset
school_data_complete_df = pd.merge(student_data_df, school_data_df, how="left", on=["school_name", "school_name"])
school_data_complete_df.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66.0,79.0,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94.0,61.0,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90.0,60.0,0,District,2917,1910635
3,3,Richard Scott,M,12th,Huang High School,67.0,58.0,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97.0,84.0,0,District,2917,1910635


In [22]:
# Calculate the Totals (Schools and Students)
school_count = len(school_data_complete_df["school_name"].unique())
student_count = school_data_complete_df["Student ID"].count()

# Calculate the Total Budget
total_budget = school_data_df["budget"].sum()

In [23]:
# Calculate the Average Scores using the "clean_student_data".
average_reading_score = school_data_complete_df["reading_score"].mean()
average_math_score = school_data_complete_df["math_score"].mean()

In [24]:
# Step 1. Get the number of students that are in ninth grade at Thomas High School.
# These students have no grades. 


# Get the total student count 
student_count = school_data_complete_df["Student ID"].count()


# Step 2. Subtract the number of students that are in ninth grade at 
# Thomas High School from the total student count to get the new total student count.


In [25]:
# Calculate the passing rates using the "clean_student_data".
passing_math_count = school_data_complete_df[(school_data_complete_df["math_score"] >= 70)].count()["student_name"]
passing_reading_count = school_data_complete_df[(school_data_complete_df["reading_score"] >= 70)].count()["student_name"]

In [26]:
# Step 3. Calculate the passing percentages with the new total student count.


In [27]:
# Calculate the students who passed both reading and math.
passing_math_reading = school_data_complete_df[(school_data_complete_df["math_score"] >= 70)
                                               & (school_data_complete_df["reading_score"] >= 70)]

# Calculate the number of students that passed both reading and math.
overall_passing_math_reading_count = passing_math_reading["student_name"].count()


# Step 4.Calculate the overall passing percentage with new total student count.


In [28]:
# Create a DataFrame
district_summary_df = pd.DataFrame(
          [{"Total Schools": school_count, 
          "Total Students": student_count, 
          "Total Budget": total_budget,
          "Average Math Score": average_math_score, 
          "Average Reading Score": average_reading_score,
          "% Passing Math": passing_math_percentage,
         "% Passing Reading": passing_reading_percentage,
        "% Overall Passing": overall_passing_percentage}])



# Format the "Total Students" to have the comma for a thousands separator.
district_summary_df["Total Students"] = district_summary_df["Total Students"].map("{:,}".format)
# Format the "Total Budget" to have the comma for a thousands separator, a decimal separator and a "$".
district_summary_df["Total Budget"] = district_summary_df["Total Budget"].map("${:,.2f}".format)
# Format the columns.
district_summary_df["Average Math Score"] = district_summary_df["Average Math Score"].map("{:.1f}".format)
district_summary_df["Average Reading Score"] = district_summary_df["Average Reading Score"].map("{:.1f}".format)
district_summary_df["% Passing Math"] = district_summary_df["% Passing Math"].map("{:.1f}".format)
district_summary_df["% Passing Reading"] = district_summary_df["% Passing Reading"].map("{:.1f}".format)
district_summary_df["% Overall Passing"] = district_summary_df["% Overall Passing"].map("{:.1f}".format)

# Display the data frame
district_summary_df

NameError: name 'passing_math_percentage' is not defined

##  School Summary

In [ ]:
# Determine the School Type
per_school_types = school_data_df.set_index(["school_name"])["type"]

# Calculate the total student count.
per_school_counts = school_data_complete_df["school_name"].value_counts()

# Calculate the total school budget and per capita spending
per_school_budget = school_data_complete_df.groupby(["school_name"]).mean()["budget"]
# Calculate the per capita spending.
per_school_capita = per_school_budget / per_school_counts

# Calculate the average test scores.
per_school_math = school_data_complete_df.groupby(["school_name"]).mean()["math_score"]
per_school_reading = school_data_complete_df.groupby(["school_name"]).mean()["reading_score"]

# Calculate the passing scores by creating a filtered DataFrame.
per_school_passing_math = school_data_complete_df[(school_data_complete_df["math_score"] >= 70)]
per_school_passing_reading = school_data_complete_df[(school_data_complete_df["reading_score"] >= 70)]

# Calculate the number of students passing math and passing reading by school.
per_school_passing_math = per_school_passing_math.groupby(["school_name"]).count()["student_name"]
per_school_passing_reading = per_school_passing_reading.groupby(["school_name"]).count()["student_name"]

# Calculate the percentage of passing math and reading scores per school.
per_school_passing_math = per_school_passing_math / per_school_counts * 100
per_school_passing_reading = per_school_passing_reading / per_school_counts * 100

# Calculate the students who passed both reading and math.
per_passing_math_reading = school_data_complete_df[(school_data_complete_df["reading_score"] >= 70)
                                               & (school_data_complete_df["math_score"] >= 70)]

# Calculate the number of students passing math and passing reading by school.
per_passing_math_reading = per_passing_math_reading.groupby(["school_name"]).count()["student_name"]

# Calculate the percentage of passing math and reading scores per school.
per_overall_passing_percentage = per_passing_math_reading / per_school_counts * 100

In [ ]:
# Create the DataFrame
per_school_summary_df = pd.DataFrame({
    "School Type": per_school_types,
    "Total Students": per_school_counts,
    "Total School Budget": per_school_budget,
    "Per Student Budget": per_school_capita,
    "Average Math Score": per_school_math,
    "Average Reading Score": per_school_reading,
    "% Passing Math": per_school_passing_math,
    "% Passing Reading": per_school_passing_reading,
    "% Overall Passing": per_overall_passing_percentage})


# per_school_summary_df.head()

In [ ]:
# Format the Total School Budget and the Per Student Budget
per_school_summary_df["Total School Budget"] = per_school_summary_df["Total School Budget"].map("${:,.2f}".format)
per_school_summary_df["Per Student Budget"] = per_school_summary_df["Per Student Budget"].map("${:,.2f}".format)

# Display the data frame
per_school_summary_df

In [ ]:
# Step 5.  Get the number of 10th-12th graders from Thomas High School (THS).


In [ ]:
# Step 6. Get all the students passing math from THS


In [ ]:
# Step 7. Get all the students passing reading from THS


In [ ]:
# Step 8. Get all the students passing math and reading from THS


In [ ]:
# Step 9. Calculate the percentage of 10th-12th grade students passing math from Thomas High School. 


In [ ]:
# Step 10. Calculate the percentage of 10th-12th grade students passing reading from Thomas High School.


In [ ]:
# Step 11. Calculate the overall passing percentage of 10th-12th grade from Thomas High School. 



In [ ]:
# Step 12. Replace the passing math percent for Thomas High School in the per_school_summary_df.


In [ ]:
# Step 13. Replace the passing reading percentage for Thomas High School in the per_school_summary_df.


In [ ]:
# Step 14. Replace the overall passing percentage for Thomas High School in the per_school_summary_df.


In [ ]:
# per_school_summary_df

## High and Low Performing Schools 

In [ ]:
# Sort and show top five schools.


In [ ]:
# Sort and show top five schools.


## Math and Reading Scores by Grade

In [ ]:
# Create a Series of scores by grade levels using conditionals.


# Group each school Series by the school name for the average math score.


# Group each school Series by the school name for the average reading score.


In [ ]:
# Combine each Series for average math scores by school into single data frame.


In [ ]:
# Combine each Series for average reading scores by school into single data frame.


In [ ]:
# Format each grade column.


In [ ]:
# Remove the index.


# Display the data frame


In [ ]:
## Remove the index.


# Display the data frame


## Scores by School Spending

In [ ]:
# Establish the spending bins and group names.


# Categorize spending based on the bins.


In [ ]:
# Calculate averages for the desired columns. 


In [ ]:
# Create the DataFrame


In [ ]:
# Format the DataFrame 


## Scores by School Size

In [ ]:
# Establish the bins.

# Categorize spending based on the bins.


In [ ]:
# Calculate averages for the desired columns. 


In [ ]:
# Assemble into DataFrame. 


In [ ]:
# Format the DataFrame  


## Scores by School Type

In [ ]:
# Calculate averages for the desired columns. 


In [ ]:
# Assemble into DataFrame. 


In [ ]:
# # Format the DataFrame 


In [ ]:


import pandas as pd

In [ ]:
school_data_df=pd.read_csv("Resources/schools_complete.csv")
students_complete_df=pd.read_csv("Resources/students_complete.csv")

In [ ]:
students_complete_df.head()


In [ ]:
prefixes_suffixes = ["Dr. ", "Mr. ","Ms. ", "Mrs. ", "Miss ", " MD", " DDS", " DVM", " PhD"]
for name in prefixes_suffixes:
    students_complete_df["student_name"] = students_complete_df["student_name"].str.replace(name,"")
    
students_complete_df

In [ ]:
students_complete_df.head()

In [ ]:
student_data_df=students_complete_df


In [ ]:
school_data_df.head()

In [ ]:
school_data_complete_df=pd.merge(student_data_df,school_data_df,on=['school_name','school_name'])

In [ ]:
school_data_complete_df.head()

In [ ]:
school_data_df.rename(columns={'school_name1':'school_name'})

In [ ]:
student_count = school_data_complete_df.count()
student_count

In [ ]:
school_data_complete_df['Student ID'].count()

In [ ]:
school_count = school_data_df["school_name"].count()
school_count

In [ ]:
@school_count = school_data_complete_df["school_name"].unique()


In [ ]:
total_budget=school_data_df['budget'].sum()
total_budget

In [ ]:
#reading_score 	math_score
average_reading_score=school_data_complete_df['reading_score'].mean()
average_reading_score

In [ ]:
average_math_score=school_data_complete_df['math_score'].mean()
average_math_score

In [ ]:
passing_math=school_data_complete_df['math_score']>=70
passing_reading=school_data_complete_df['reading_score']>=70
passing_math

In [ ]:
passing_math=school_data_complete_df[passing_math]

In [ ]:
passing_reading

In [ ]:
passing_reading = school_data_complete_df[school_data_complete_df["reading_score"] >= 70]
passing_reading

In [ ]:
passing_math = school_data_complete_df[school_data_complete_df["math_score"] >= 70]
passing_math.head()

In [ ]:
passing_math_count=passing_math['student_name'].count()
passing_math_count

In [ ]:
passing_reading_count=passing_reading['student_name'].count()
passing_reading_count

In [ ]:
student_count=school_data_complete_df["Student ID"].count()

In [ ]:
student_count

In [ ]:
student_count.dtype

In [ ]:
passing_math_percentage=passing_math_count/float(student_count)*100
passing_math_percentage.dtype

In [ ]:
passing_reading__percentage=passing_reading_count/float(student_count)*100
passing_reading__percentage

In [ ]:
#passin_math_reading=school_data_complete_df[school_data_complete_df["math_score"] >= 70 & ['reading_score']>=70]

In [ ]:
passing_math_reading=school_data_complete_df[(school_data_complete_df["math_score"] >= 70)  & (school_data_complete_df["reading_score"] >=70)]
passing_math_reading.student_name.count()

In [ ]:
overall_passing_math_reading_count = passing_math_reading["student_name"].count()
overall_passing_math_reading_count

In [ ]:
overall_passing_percentage=overall_passing_math_reading_count/float(student_count)*100
overall_passing_percentage

In [ ]:
student_count

In [ ]:
district_summary_df=pd.DataFrame([{"Total Schools":school_count,"Total Students":student_count,"Total Budget":total_budget ,
                                  "Average Math Score": average_math_score,
          "Average Reading Score": average_reading_score,
          "% Passing Math": passing_math_percentage,
         "% Passing Reading": passing_reading__percentage,
        "% Overall Passing": overall_passing_percentage}])
district_summary_df

In [ ]:
district_summary_df["Average Math Score"]=district_summary_df["Average Math Score"].map("{:.2f}".format)

In [ ]:
district_summary_df

In [ ]:
district_summary_df["Total Students"]=district_summary_df["Total Students"].map("{:,}".format)
district_summary_df

In [ ]:
district_summary_df["Total Budget"]=district_summary_df["Total Budget"].map("${:,.2f}".format)
district_summary_df

In [ ]:
district_summary_df["Average Reading Score"]=district_summary_df["Average Reading Score"].map("{:.2f}".format)

district_summary_df                                                                             

In [ ]:
#district_summary_df["% Passing Math"]=district_summary_df["% Passing Math"].map("{:.0f}".format)
#district_summary_df["% Passing Math"] = district_summary_df["% Passing Math"].map("{:.0f}".format)
district_summary_df.dtypes

In [ ]:
# Format the columns.
district_summary_df["Average Math Score"] = district_summary_df["Average Math Score"].map("{:.1f}".format)

district_summary_df["Average Reading Score"] = district_summary_df["Average Reading Score"].map("{:.1f}".format)

district_summary_df["% Passing Math"] = district_summary_df["% Passing Math"].map("{:.0f}".format)

district_summary_df["% Passing Reading"] = district_summary_df["% Passing Reading"].map("{:.0f}".format)

district_summary_df["% Overall Passing"] = district_summary_df["% Overall Passing"].map("{:.0f}".format)

In [ ]:
district_summary_df

In [ ]:
new_order=["Total Schools", "Total Students", "Total Budget","Average Math Score", "Average Reading Score", "% Passing Math", "% Passing Reading", "% Overall Passing"]
district_summary_df=district_summary_df[new_order]
district_summary_df

In [ ]:
#district_summary_df=district_summary_df["Total Schools", "Total Students", "Total Budget","Average Math Score", "Average Reading Score", "% Passing Math", "% Passing Reading", "% Overall Passing"]

In [ ]:
school_data_complete_df.head()


In [ ]:
school_data_complete_df.index

In [ ]:
school_data_df.head()

In [ ]:
per_school_types =school_data_df.set_index(["school_name"])["type"]
per_school_types

In [ ]:
df = pd.DataFrame(per_school_types)
df

In [ ]:
per_school_counts = school_data_df["size"]
per_school_counts

In [ ]:
per_school_counts = school_data_df.set_index(["school_name"])["size"]
per_school_counts

In [ ]:
school_data_complete_df.head()

In [ ]:
school_data_complete_df.value_counts("school_name")

In [ ]:
per_school_counts = school_data_complete_df["school_name"].value_counts()
per_school_counts

In [ ]:
per_school_budget=school_data_df.set_index(["school_name"])["budget"]
per_school_budget

In [ ]:
per_school_capita = per_school_budget / per_school_counts
per_school_capita

In [ ]:
student_data_df.head()

In [ ]:
average=student_data_df.set_index(["school_name"])['reading_score']
average

In [ ]:
per_school_reading=school_data_complete_df.groupby(['school_name']).mean()["reading_score"]
per_school_reading

In [ ]:
per_school_math = school_data_complete_df.groupby(["school_name"]).mean()["math_score"]
per_school_math

In [ ]:
per_school_passing_math = school_data_complete_df[(school_data_complete_df["math_score"] >= 70)]

per_school_passing_reading = school_data_complete_df[(school_data_complete_df["reading_score"] >= 70)]
per_passing_math_reading =school_data_complete_df[(school_data_complete_df["reading_score"] >= 70)&(school_data_complete_df["math_score"] >= 70)]

In [ ]:
per_school_passing_math=per_school_passing_math.groupby(["school_name"]).count()["student_name"]

In [ ]:
per_school_passing_reading=per_school_passing_reading.groupby(["school_name"]).count()['student_name']
per_passing_math_reading=per_passing_math_reading.groupby(["school_name"]).count()['student_name']

In [ ]:
per_school_passing_math_percentage=per_school_passing_math/per_school_counts*100

per_school_passing_math_percentage=per_school_passing_math_percentage.map("{:.2f}".format)
per_school_passing_reading_percentage=per_school_passing_reading/per_school_counts*100

per_school_passing_reading_percentage=per_school_passing_reading_percentage.map("{:.2f}".format)
per_passing_math_reading_percentage=per_passing_math_reading/per_school_counts*100
per_passing_math_reading_percentage=per_passing_math_reading_percentage.map("{:.0f}".format)
per_passing_math_reading_percentage

In [ ]:
per_school_counts

In [ ]:

#per_passing_math_reading_percentage=per_passing_math_reading_percentage.map("{:2f}".format)


In [ ]:
per_passing_math_reading_percentage

In [ ]:
per_school_summary_df = pd.DataFrame({
             "School Type": per_school_types,
             "Total Students": per_school_counts,
             "Total School Budget": per_school_budget,
             "Per Student Budget": per_school_capita,
             "Average Math Score": per_school_math,
           "Average Reading Score": per_school_reading,
           "% Passing Math": per_school_passing_math_percentage,
           "% Passing Reading": per_school_passing_reading_percentage,
           "% Overall Passing": per_passing_math_reading_percentage})
per_school_summary_df.head()

In [ ]:
per_school_summary_df["Total School Budget"]=per_school_summary_df["Total School Budget"].map("${:,.2f}".format)
per_school_summary_df["Average Math Score"]=per_school_summary_df["Average Math Score"].map("{:.2f}".format)
per_school_summary_df["Average Reading Score"]=per_school_summary_df["Average Reading Score"].map("{:.2f}".format)
per_school_summary_df["Per Student Budget"]=per_school_summary_df["Per Student Budget"].map("${:.2f}".format)




In [ ]:
per_school_summary_df.head()


In [ ]:
top_schools=per_school_summary_df.sort_values("% Overall Passing",ascending=False)
top_schools.head()

In [ ]:
bottom_schools=per_school_summary_df.sort_values(["% Overall Passing"])
bottom_schools.head()

In [ ]:
school_data_complete_df.head()

In [ ]:
grade=school_data_complete_df.groupby("school_name")["grade"]

In [ ]:
grade

In [ ]:
ninth_graders = school_data_complete_df[(school_data_complete_df["grade"] == "9th")]

tenth_graders = school_data_complete_df[(school_data_complete_df["grade"] == "10th")]

eleventh_graders = school_data_complete_df[(school_data_complete_df["grade"] == "11th")]

twelfth_graders = school_data_complete_df[(school_data_complete_df["grade"] == "12th")]

In [ ]:
ninth_grade_math_scores = ninth_graders.groupby(["school_name"]).mean()["math_score"]

tenth_grade_math_scores = tenth_graders.groupby(["school_name"]).mean()["math_score"]

eleventh_grade_math_scores = eleventh_graders.groupby(["school_name"]).mean()["math_score"]

twelfth_grade_math_scores = twelfth_graders.groupby(["school_name"]).mean()["math_score"]

In [ ]:
eleventh_grade_math_scores

In [ ]:
ninth_grade_reading_scores = ninth_graders.groupby(["school_name"]).mean()["reading_score"]

tenth_grade_reading_scores = tenth_graders.groupby(["school_name"]).mean()["reading_score"]

eleventh_grade_reading_scores = eleventh_graders.groupby(["school_name"]).mean()["reading_score"]

twelfth_grade_reading_scores = twelfth_graders.groupby(["school_name"]).mean()["reading_score"]


In [ ]:
math_scores_by_grade = pd.DataFrame({
               "9th": ninth_grade_math_scores,
               "10th": tenth_grade_math_scores,
               "11th": eleventh_grade_math_scores,
               "12th": twelfth_grade_math_scores})

math_scores_by_grade["9th"]=math_scores_by_grade["9th"].map("{:.2f}".format)
math_scores_by_grade["10th"]=math_scores_by_grade["10th"].map("{:.2f}".format)
math_scores_by_grade["11th"]=math_scores_by_grade["11th"].map("{:.2f}".format)
math_scores_by_grade["12th"]=math_scores_by_grade["12th"].map("{:.2f}".format)

math_scores_by_grade.head()
                      
                     

In [ ]:
reading_scores_by_grade = pd.DataFrame({
              "9th": ninth_grade_reading_scores,
              "10th": tenth_grade_reading_scores,
              "11th": eleventh_grade_reading_scores,
              "12th": twelfth_grade_reading_scores})

reading_scores_by_grade["9th"]=reading_scores_by_grade["9th"].map("{:.2f}".format)
reading_scores_by_grade["10th"]=reading_scores_by_grade["10th"].map("{:.2f}".format)
reading_scores_by_grade["11th"]=reading_scores_by_grade["11th"].map("{:.2f}".format)
reading_scores_by_grade["12th"]=reading_scores_by_grade["12th"].map("{:.2f}".format)

reading_scores_by_grade.head()

In [ ]:
math_scores_by_grade.index.name= None
reading_scores_by_grade.index.name =None

In [ ]:
reading_scores_by_grade.head()

In [ ]:
math_scores_by_grade

In [ ]:
per_school_capita

In [ ]:
per_school_capita.describe()

In [ ]:
spending_bins = [0, 585, 630, 645, 675]

In [ ]:
spending_bins

In [ ]:
pd.cut(per_school_capita, spending_bins)

In [ ]:
per_school_capita.groupby(pd.cut(per_school_capita, spending_bins)).count()

In [ ]:
spending_bins = [0, 585, 630, 645, 675]
group_names = ["<$584", "$585-629", "$630-644", "$645-675"]

In [ ]:
per_school_summary_df.head()

In [ ]:
per_school_summary_df["Spending Ranges (Per Student)"] = pd.cut(per_school_capita, spending_bins, labels=group_names)

per_school_summary_df

In [ ]:
per_school_summary_df["Average Math Score"]=pd.to_numeric(per_school_summary_df["Average Math Score"])
per_school_summary_df["Average Reading Score"]=pd.to_numeric(per_school_summary_df["Average Reading Score"])
per_school_summary_df["% Passing Math"]=pd.to_numeric(per_school_summary_df["% Passing Math"])
per_school_summary_df["% Passing Reading"]=pd.to_numeric(per_school_summary_df["% Passing Reading"])
per_school_summary_df["% Overall Passing"]=pd.to_numeric(per_school_summary_df["% Overall Passing"])


In [ ]:
spending_math_scores = per_school_summary_df.groupby(["Spending Ranges (Per Student)"]).mean()["Average Math Score"]

spending_reading_scores = per_school_summary_df.groupby(["Spending Ranges (Per Student)"]).mean()["Average Reading Score"]

spending_passing_math = per_school_summary_df.groupby(["Spending Ranges (Per Student)"]).mean()["% Passing Math"]

spending_passing_reading = per_school_summary_df.groupby(["Spending Ranges (Per Student)"]).mean()["% Passing Reading"]

overall_passing_spending = per_school_summary_df.groupby(["Spending Ranges (Per Student)"]).mean()["% Overall Passing"]

In [ ]:
overall_passing_spending

In [ ]:
spending_summary_df = pd.DataFrame({
          "Average Math Score" : spending_math_scores,
          "Average Reading Score": spending_reading_scores,
          "% Passing Math": spending_passing_math,
          "% Passing Reading": spending_passing_reading,
          "% Overall Passing": overall_passing_spending})

spending_summary_df


In [ ]:
spending_summary_df["Average Math Score"]=spending_summary_df["Average Math Score"].map("{:.1f}".format)
spending_summary_df["Average Reading Score"]=spending_summary_df["Average Reading Score"].map("{:.1f}".format)
spending_summary_df["% Passing Math"] = spending_summary_df["% Passing Math"].map("{:.0f}".format)

spending_summary_df["% Passing Reading"] = spending_summary_df["% Passing Reading"].map("{:.0f}".format)

spending_summary_df["% Overall Passing"] = spending_summary_df["% Overall Passing"].map("{:.0f}".format)


In [ ]:
spending_summary_df

In [ ]:
per_school_counts.describe()


In [ ]:
school_size=[0,1000,1999,5000]
group_names = ["Small", "Medium", "Large"]
pd.cut(per_school_summary_df, school_size)

In [ ]:
per_school_summary_df["School size)"] = pd.cut(per_school_summary_df, school_size, labels=group_names)

per_school_summary_df

In [ ]:
size_bins = [0, 1000, 2000, 5000]
group_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]
per_school_summary_df["School Size"] = pd.cut(per_school_summary_df["Total Students"], size_bins, labels=group_names)

per_school_summary_df.head()

In [ ]:
size_math_scores=per_school_summary_df.groupby('School Size').mean()["Average Math Score"]
size_reading_scores = per_school_summary_df.groupby(["School Size"]).mean()["Average Reading Score"]

size_passing_math = per_school_summary_df.groupby(["School Size"]).mean()["% Passing Math"]

size_passing_reading = per_school_summary_df.groupby(["School Size"]).mean()["% Passing Reading"]

size_overall_passing = per_school_summary_df.groupby(["School Size"]).mean()["% Overall Passing"]

In [ ]:
size_summary_df=pd.DataFrame({
          "Average Math Score" : size_math_scores,
          "Average Reading Score": size_reading_scores,
          "% Passing Math": size_passing_math ,
          "% Passing Reading": size_passing_reading,
          "% Overall Passing": size_overall_passing})
size_summary_df

In [ ]:
# Formatting.
size_summary_df["Average Math Score"] = size_summary_df["Average Math Score"].map("{:.1f}".format)

size_summary_df["Average Reading Score"] = size_summary_df["Average Reading Score"].map("{:.1f}".format)

size_summary_df["% Passing Math"] = size_summary_df["% Passing Math"].map("{:.0f}".format)

size_summary_df["% Passing Reading"] = size_summary_df["% Passing Reading"].map("{:.0f}".format)

size_summary_df["% Overall Passing"] = size_summary_df["% Overall Passing"].map("{:.0f}".format)

size_summary_df

In [ ]:
per_school_summary_df.head()


In [ ]:
type_math_scores=per_school_summary_df.groupby('School Type').mean()["Average Math Score"]
type_reading_scores = per_school_summary_df.groupby(["School Type"]).mean()["Average Reading Score"]

type_passing_math = per_school_summary_df.groupby(["School Type"]).mean()["% Passing Math"]

type_passing_reading = per_school_summary_df.groupby(["School Type"]).mean()["% Passing Reading"]

type_overall_passing = per_school_summary_df.groupby(["School Type"]).mean()["% Overall Passing"]

type_summary_df=pd.DataFrame({
          "Average Math Score" : type_math_scores,
          "Average Reading Score": type_reading_scores,
          "% Passing Math": type_passing_math ,
          "% Passing Reading": type_passing_reading,
          "% Overall Passing": type_overall_passing})
type_summary_df.dtypes

In [ ]:
type_summary_df["Average Math Score"] = type_summary_df["Average Math Score"].map("{:.1f}".format)

type_summary_df["Average Reading Score"] = type_summary_df["Average Reading Score"].map("{:.1f}".format)

type_summary_df["% Passing Math"] = type_summary_df["% Passing Math"].map("{:.0f}".format)

type_summary_df["% Passing Reading"] = type_summary_df["% Passing Reading"].map("{:.0f}".format)

type_summary_df["% Overall Passing"] = type_summary_df["% Overall Passing"].map("{:.0f}".format)


In [ ]:
type_summary_df.dtypes

In [ ]:
type_summary_df